In [1]:
import pandas as pd
import glob as gb

#Load dataset
data=r"D:\RealTime_Alert_Analysis\dataset"

#club all files into one
files=gb.glob(f"{data}/*.pcap_ISCX.csv")
df_list = [pd.read_csv(f) for f in files]
df = pd.concat(df_list, ignore_index=True)

# print(df.columns.tolist())



In [2]:
#Encoding labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[' Label'] = le.fit_transform(df[' Label'])

# print("Successfully! Labels encoded.")
# print(dict(zip(le.classes_, le.transform(le.classes_))))




In [3]:
#Split dataset into features and labels

X = df.drop(' Label', axis=1)
y = df[' Label']

# print(X,y)
# print(X.shape, y.shape)

import numpy as np

# Replace inf and -inf with NaN
X = X.replace([np.inf, -np.inf], np.nan)
X = X.dropna()
y = y.loc[X.index]

#Splitting dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

# print("Training samples:", X_train.shape[0])
# print("Testing samples:", x_test.shape[0])

In [4]:
#Training the model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

print("✅ Random Forest model trained successfully!")


✅ Random Forest model trained successfully!


In [6]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9982124418292149


d:\RealTime_Alert_Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    454265
           1       0.96      0.37      0.53       391
           2       1.00      1.00      1.00     25605
           3       1.00      0.97      0.99      2059
           4       1.00      1.00      1.00     46025
           5       0.99      0.98      0.99      1100
           6       1.00      0.99      1.00      1159
           7       1.00      1.00      1.00      1587
           8       1.00      1.00      1.00         2
           9       1.00      0.29      0.44         7
          10       0.99      1.00      1.00     31761
          11       1.00      0.99      1.00      1180
          12       0.66      0.81      0.73       301
          13       0.00      0.00      0.00         4
          14       1.00      0.01      0.02       130

    accuracy                           1.00    565576
   macro avg       0.91      0.76      0.78    565576
weighted avg       1.00   

d:\RealTime_Alert_Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\RealTime_Alert_Analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv(r"D:\RealTime_Alert_Analysis\dataset\Network_dataset_1.csv", low_memory=False)

ip_regex = re.compile(r"^\d{1,3}(\.\d{1,3}){3}$")

print("\n🔍 Scanning dataset for non-numeric / IP-like columns...\n")

problem_cols = []
for col in df.columns:
    # Check column type and contents
    s = df[col].astype(str)
    ip_like_count = s.str.match(ip_regex).sum()
    try:
        pd.to_numeric(s, errors="raise")
    except Exception:
        # Record only if at least 1 problematic entry
        bad_examples = s[s.str.match(ip_regex) | ~s.str.match(r"^-?\d+(\.\d+)?$")].unique()[:5]
        problem_cols.append({
            "Column": col,
            "Type": df[col].dtype,
            "IP_like_values": int(ip_like_count),
            "Example_Entries": list(bad_examples)
        })

if not problem_cols:
    print("✅ All columns are clean and numeric!")
else:
    print(f"⚠ Found {len(problem_cols)} problematic columns:\n")
    for item in problem_cols:
        print(f"🧱 Column: {item['Column']}")
        print(f"   → Type: {item['Type']}")
        print(f"   → IP-like values: {item['IP_like_values']}")
        print(f"   → Example entries: {item['Example_Entries']}\n")



🔍 Scanning dataset for non-numeric / IP-like columns...

⚠ Found 29 problematic columns:

🧱 Column: src_ip
   → Type: object
   → IP-like values: 987993
   → Example entries: ['3.122.49.24', '192.168.1.79', '192.168.1.152', '192.168.1.193', '192.168.1.250']

🧱 Column: dst_ip
   → Type: object
   → IP-like values: 987993
   → Example entries: ['192.168.1.152', '192.168.1.255', '192.168.1.190', '172.217.167.67', '216.58.199.66']

🧱 Column: proto
   → Type: object
   → IP-like values: 0
   → Example entries: ['tcp', 'udp', 'icmp']

🧱 Column: service
   → Type: object
   → IP-like values: 0
   → Example entries: ['-', 'dns', 'ssl', 'dhcp', 'http']

🧱 Column: src_bytes
   → Type: object
   → IP-like values: 175
   → Example entries: ['0.0.0.0']

🧱 Column: conn_state
   → Type: object
   → IP-like values: 0
   → Example entries: ['OTH', 'S0', 'SHR', 'RSTRH', 'SH']

🧱 Column: dns_query
   → Type: object
   → IP-like values: 0
   → Example entries: ['-', '_sleep-proxy._udp.local', '_raop._tcp

In [ ]:
import pandas as pd

df = pd.read_csv(r"D:\RealTime_Alert_Analysis\dataset\Network_dataset_1.csv", low_memory=False)

# ------------------------------------------------------------------
# 🔍  Deep clean just before inference  (catch any residual IP strings)
# ------------------------------------------------------------------
ip_pattern = re.compile(r"^\d{1,3}(\.\d{1,3}){3}$")

# find any column that still contains IP-like strings
bad_cols = []
for c in X.columns:
    if X[c].astype(str).str.match(ip_pattern).any():
        bad_cols.append(c)
        # replace all IP-like entries with 0
        X[c] = X[c].astype(str).apply(lambda v: 0 if ip_pattern.match(v) else v)

# force numeric conversion one more time
X = X.apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.float32)

if bad_cols:
    st.warning(f"🧹 Sanitized residual IP-like values in: {', '.join(bad_cols)}")


🔍 Columns still containing IP-like strings (even if numeric):
src_ip       987993
dst_ip       987993
src_bytes       175
dtype: int64

🧱 src_ip: ['3.122.49.24' '192.168.1.79' '192.168.1.152' '192.168.1.193'
 '192.168.1.250' '192.168.1.133' '192.168.1.192' '192.168.1.103'
 '192.168.1.46' '192.168.1.195']

🧱 dst_ip: ['192.168.1.152' '192.168.1.255' '192.168.1.190' '172.217.167.67'
 '216.58.199.66' '224.0.0.251' '52.40.98.101' '52.35.215.194'
 '117.18.237.29' '192.168.1.192']

🧱 src_bytes: ['0.0.0.0']


In [8]:
import joblib
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

MODEL_PATH = r"D:\RealTime_Alert_Analysis\report\training_report\trained_model_balanced_final_fixed.joblib"
OUTPUT_PATH = r"D:\RealTime_Alert_Analysis\dataset\realistic_balanced_attacks.csv"

# Load model metadata
bundle = joblib.load(MODEL_PATH)
features = bundle.get("feature_names", [])
classes = [
    "BENIGN", "DDoS", "PortScan", "Botnet",
    "Infiltration", "Web Attack – Brute Force",
    "Web Attack – XSS", "Web Attack – Sql Injection",
    "Heartbleed", "ANOMALY"
]

np.random.seed(42)
data = []

# Generate synthetic data with realistic patterns
for label in classes:
    for _ in range(200):
        if label == "BENIGN":
            row = np.random.normal(0.3, 0.05, len(features))
        elif label == "DDoS":
            row = np.random.normal(0.9, 0.1, len(features))
        elif label == "PortScan":
            row = np.random.normal(0.8, 0.1, len(features))
        elif label == "Botnet":
            row = np.random.normal(0.75, 0.1, len(features))
        elif label == "Infiltration":
            row = np.random.normal(0.7, 0.1, len(features))
        elif "Web Attack" in label:
            row = np.random.normal(0.6, 0.1, len(features))
        elif label == "Heartbleed":
            row = np.random.normal(0.95, 0.05, len(features))
        elif label == "ANOMALY":
            row = np.random.normal(1.0, 0.15, len(features))
        else:
            row = np.random.normal(0.5, 0.1, len(features))
        data.append(row.tolist() + [label])

# Create DataFrame
df = pd.DataFrame(data, columns=features + ["attack_type"])
df = shuffle(df).reset_index(drop=True)
df.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Realistic test dataset created → {OUTPUT_PATH}")
print(f"Features: {len(features)} | Classes: {len(classes)} | Rows: {len(df)}")


✅ Realistic test dataset created → D:\RealTime_Alert_Analysis\dataset\realistic_balanced_attacks.csv
Features: 78 | Classes: 10 | Rows: 2000


In [15]:
import numpy as np
import pandas as pd
import os

# ======================================
# CONFIGURATION
# ======================================
OUTPUT_PATH = r"D:\RealTime_Alert_Analysis\notebook\model_aligned_network_dataset.csv"
NUM_SAMPLES = 5000
NUM_FEATURES = 78
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ======================================
# ATTACK LABELS & RISK LEVELS
# ======================================
attack_types = {
    "BENIGN": "Low",
    "Normal": "Low",
    "Botnet": "Medium",
    "Web Attack – XSS": "Medium",
    "Web Attack – Brute Force": "High",
    "Infiltration": "High",
    "PortScan": "High",
    "DDoS": "Critical",
    "Heartbleed": "Critical",
    "ANOMALY": "Critical"
}

distribution = {
    "BENIGN": 0.25,
    "Normal": 0.10,
    "Botnet": 0.10,
    "Web Attack – XSS": 0.08,
    "Web Attack – Brute Force": 0.08,
    "Infiltration": 0.07,
    "PortScan": 0.10,
    "DDoS": 0.10,
    "Heartbleed": 0.07
}
distribution = {k: v / sum(distribution.values()) for k, v in distribution.items()}

# ======================================
# CORRELATED FEATURE GENERATOR
# ======================================
def generate_correlated_data(base_mean, scale, correlation_strength, n_samples):
    """
    Generate correlated network-like features with a shared signal and controlled noise.
    """
    base_signal = np.random.normal(base_mean, scale, (n_samples, 1))
    correlated = base_signal + np.random.normal(0, scale * (1 - correlation_strength), (n_samples, NUM_FEATURES))
    correlated = np.clip(correlated, 0, None)  # Ensure no negative features
    return correlated

# ======================================
# ATTACK BEHAVIOR PROFILES
# ======================================
def generate_attack_profile(label, n):
    """
    Create distinct, correlated, semi-realistic features per attack type.
    """
    if label == "BENIGN":
        return generate_correlated_data(10, 2, 0.9, n)
    elif label == "Normal":
        return generate_correlated_data(20, 5, 0.85, n)
    elif label == "Botnet":
        return generate_correlated_data(50, 15, 0.8, n)
    elif label == "Web Attack – XSS":
        return generate_correlated_data(80, 20, 0.75, n)
    elif label == "Web Attack – Brute Force":
        return generate_correlated_data(150, 50, 0.7, n)
    elif label == "Infiltration":
        return generate_correlated_data(300, 100, 0.6, n)
    elif label == "PortScan":
        return generate_correlated_data(500, 150, 0.6, n)
    elif label == "DDoS":
        return generate_correlated_data(1000, 300, 0.5, n)
    elif label == "Heartbleed":
        return generate_correlated_data(2000, 600, 0.4, n)
    else:  # ANOMALY
        arr = np.random.normal(5000, 2000, (n, NUM_FEATURES))
        arr += np.random.uniform(0, 10000, arr.shape)  # wild spikes
        return np.abs(arr)

# ======================================
# BUILD DATASET
# ======================================
frames = []
for attack, prob in distribution.items():
    n = int(NUM_SAMPLES * prob)
    data = generate_attack_profile(attack, n)
    df_temp = pd.DataFrame(data, columns=[f"feature_{i+1}" for i in range(NUM_FEATURES)])
    df_temp["attack_type"] = attack
    frames.append(df_temp)

# Add anomalies
num_anomalies = int(NUM_SAMPLES * 0.1)
anom_data = generate_attack_profile("ANOMALY", num_anomalies)
df_anom = pd.DataFrame(anom_data, columns=[f"feature_{i+1}" for i in range(NUM_FEATURES)])
df_anom["attack_type"] = "ANOMALY"
frames.append(df_anom)

df = pd.concat(frames, ignore_index=True)
df["Label"] = df["attack_type"]
df["Risk_Level"] = df["attack_type"].map(attack_types)

# Shuffle dataset
df = df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Save dataset
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df.to_csv(OUTPUT_PATH, index=False)

print("✅ Model-aligned dataset generated successfully!")
print(f"📁 Saved to: {OUTPUT_PATH}")
print("📊 Attack distribution:")
print(df['attack_type'].value_counts())


✅ Model-aligned dataset generated successfully!
📁 Saved to: D:\RealTime_Alert_Analysis\notebook\model_aligned_network_dataset.csv
📊 Attack distribution:
attack_type
BENIGN                      1315
Botnet                       526
PortScan                     526
DDoS                         526
Normal                       526
ANOMALY                      500
Web Attack – Brute Force     421
Web Attack – XSS             421
Heartbleed                   368
Infiltration                 368
Name: count, dtype: int64


In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

OUTPUT_PATH = r"D:\RealTime_Alert_Analysis\notebook\realistic_self_scaled_network_dataset.csv"
NUM_SAMPLES = 5000
NUM_FEATURES = 78
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

attack_types = {
    "BENIGN": "Low",
    "Normal": "Low",
    "Botnet": "Medium",
    "Web Attack – XSS": "Medium",
    "Web Attack – Brute Force": "High",
    "Infiltration": "High",
    "PortScan": "High",
    "DDoS": "Critical",
    "Heartbleed": "Critical",
    "ANOMALY": "Critical"
}

distribution = {
    "BENIGN": 0.25,
    "Normal": 0.10,
    "Botnet": 0.10,
    "Web Attack – XSS": 0.08,
    "Web Attack – Brute Force": 0.08,
    "Infiltration": 0.07,
    "PortScan": 0.10,
    "DDoS": 0.10,
    "Heartbleed": 0.07
}
distribution = {k: v / sum(distribution.values()) for k, v in distribution.items()}

def generate_attack_profile(label, n):
    base_mean, base_std = {
        "BENIGN": (10, 3),
        "Normal": (20, 6),
        "Botnet": (50, 10),
        "Web Attack – XSS": (80, 15),
        "Web Attack – Brute Force": (150, 40),
        "Infiltration": (300, 70),
        "PortScan": (500, 100),
        "DDoS": (900, 200),
        "Heartbleed": (2000, 400)
    }.get(label, (1000, 250))
    
    data = np.random.normal(base_mean, base_std, (n, NUM_FEATURES))
    data = np.clip(data, 0, None)
    return data

frames = []
for attack, prob in distribution.items():
    n = int(NUM_SAMPLES * prob)
    df_part = pd.DataFrame(
        generate_attack_profile(attack, n),
        columns=[f"feature_{i+1}" for i in range(NUM_FEATURES)]
    )
    df_part["attack_type"] = attack
    frames.append(df_part)

# Add anomalies (wild random noise)
anom_n = int(NUM_SAMPLES * 0.1)
anom_data = np.abs(np.random.normal(5000, 2000, (anom_n, NUM_FEATURES)))
df_anom = pd.DataFrame(anom_data, columns=[f"feature_{i+1}" for i in range(NUM_FEATURES)])
df_anom["attack_type"] = "ANOMALY"
frames.append(df_anom)

df = pd.concat(frames, ignore_index=True)
df["Risk_Level"] = df["attack_type"].map(attack_types)
df["Label"] = df["attack_type"]

# ✅ Scale numerically for model input consistency
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df.iloc[:, :-3]), columns=df.columns[:-3])
df_scaled["attack_type"] = df["attack_type"]
df_scaled["Risk_Level"] = df["Risk_Level"]
df_scaled["Label"] = df["Label"]

# Shuffle dataset
df_scaled = df_scaled.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df_scaled.to_csv(OUTPUT_PATH, index=False)

print("✅ Realistic self-scaled dataset generated successfully!")
print(f"📁 Saved to: {OUTPUT_PATH}")
print("📊 Attack distribution:")
print(df_scaled['attack_type'].value_counts())


✅ Realistic self-scaled dataset generated successfully!
📁 Saved to: D:\RealTime_Alert_Analysis\notebook\realistic_self_scaled_network_dataset.csv
📊 Attack distribution:
attack_type
BENIGN                      1315
Botnet                       526
PortScan                     526
DDoS                         526
Normal                       526
ANOMALY                      500
Web Attack – Brute Force     421
Web Attack – XSS             421
Heartbleed                   368
Infiltration                 368
Name: count, dtype: int64


In [18]:
import joblib
import numpy as np

SCALER_PATH = r"D:\RealTime_Alert_Analysis\report\training_report\standard_scaler.joblib"

try:
    scaler = joblib.load(SCALER_PATH)
    print("✅ Scaler loaded successfully!")
    print(f"Mean shape: {scaler.mean_.shape}")
    print(f"Var shape: {scaler.var_.shape}\n")
    print("📊 Example feature statistics (first 10):")
    for i in range(min(10, len(scaler.mean_))):
        print(f"Feature {i+1}: mean={scaler.mean_[i]:.4f}, std={np.sqrt(scaler.var_[i]):.4f}")
except Exception as e:
    print(f"❌ Error loading scaler: {e}")


✅ Scaler loaded successfully!
Mean shape: (78,)
Var shape: (78,)

📊 Example feature statistics (first 10):
Feature 1: mean=0.0000, std=0.0000
Feature 2: mean=0.0000, std=0.0000
Feature 3: mean=0.0000, std=0.0000
Feature 4: mean=0.0000, std=0.0000
Feature 5: mean=0.0000, std=0.0000
Feature 6: mean=0.0000, std=0.0000
Feature 7: mean=0.0000, std=0.0000
Feature 8: mean=0.0000, std=0.0000
Feature 9: mean=0.0000, std=0.0000
Feature 10: mean=0.0000, std=0.0000


In [20]:
import joblib

MODEL_PATH = r"D:\RealTime_Alert_Analysis\report\training_report\trained_model_balanced_final_fixed.joblib"
model = joblib.load(MODEL_PATH)
clf = model.get("classifier", None)
le = model.get("label_encoder", None)

print("✅ Model components:")
print(model.keys())

if le:
    print("\n🎯 LabelEncoder classes:")
    print(le.classes_)

if clf:
    print(f"\n🌲 RandomForest info:")
    print(f"n_classes_: {getattr(clf, 'n_classes_', 'N/A')}")
    print(f"n_features_in_: {getattr(clf, 'n_features_in_', 'N/A')}")
    print(f"n_estimators: {len(clf.estimators_)}")
    if hasattr(clf, "classes_"):
        print(f"Trained classes: {clf.classes_}")


✅ Model components:
dict_keys(['classifier', 'isolation_forest', 'label_encoder', 'class_names', 'feature_names'])

🎯 LabelEncoder classes:
[0.]

🌲 RandomForest info:
n_classes_: 1
n_features_in_: 78
n_estimators: 150
Trained classes: [0]


In [21]:
import os
import pandas as pd
from glob import glob

# ======================================
# CONFIG
# ======================================
DATA_DIR = r"D:\RealTime_Alert_Analysis\dataset"   # 👈 Change if needed
OUTPUT_FILE = r"D:\RealTime_Alert_Analysis\report\training_report\column_report.txt"

# ======================================
# SCAN ALL CSV FILES
# ======================================
csv_files = glob(os.path.join(DATA_DIR, "*.csv"))
if not csv_files:
    print(f"❌ No CSV files found in {DATA_DIR}")
    exit()

report_lines = []
print(f"📦 Found {len(csv_files)} CSV files.\n")

for file in csv_files:
    try:
        df = pd.read_csv(file, nrows=5)  # read first 5 rows for speed
        cols = list(df.columns)
        report_lines.append(f"\n📁 {os.path.basename(file)} ({len(cols)} columns)\n" + "\n".join([f"  - {c}" for c in cols]))
        print(f"✅ {os.path.basename(file)} → {len(cols)} columns")
    except Exception as e:
        report_lines.append(f"\n⚠️ {os.path.basename(file)}: Error reading file → {e}")
        print(f"⚠️ Skipped {file}: {e}")

# ======================================
# FIND COMMON AND UNIQUE COLUMNS
# ======================================
all_col_sets = []
for file in csv_files:
    try:
        df = pd.read_csv(file, nrows=1)
        all_col_sets.append(set(df.columns))
    except Exception:
        continue

if all_col_sets:
    common_cols = set.intersection(*all_col_sets)
    all_cols = set.union(*all_col_sets)

    report_lines.append("\n\n===============================")
    report_lines.append(f"✅ COMMON COLUMNS ({len(common_cols)}):\n" + "\n".join(sorted(common_cols)))
    report_lines.append("\n\n===============================")
    report_lines.append(f"🧩 TOTAL UNIQUE COLUMNS ({len(all_cols)}):\n" + "\n".join(sorted(all_cols)))

# ======================================
# SAVE REPORT
# ======================================
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(report_lines))

print(f"\n📄 Column report saved to: {OUTPUT_FILE}")
print("✅ Please send me the text content of that file (or screenshot).")


📦 Found 31 CSV files.

✅ Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv → 79 columns
✅ Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv → 79 columns
✅ Friday-WorkingHours-Morning.pcap_ISCX.csv → 79 columns
✅ model_aligned_network_dataset.csv → 81 columns
✅ Network_dataset_1.csv → 46 columns
✅ Network_dataset_10.csv → 46 columns
✅ Network_dataset_11.csv → 46 columns
✅ Network_dataset_12.csv → 46 columns
✅ Network_dataset_13.csv → 46 columns
✅ Network_dataset_14.csv → 46 columns
✅ Network_dataset_15.csv → 46 columns
✅ Network_dataset_16.csv → 46 columns
✅ Network_dataset_17.csv → 46 columns
✅ Network_dataset_18.csv → 46 columns
✅ Network_dataset_19.csv → 46 columns
✅ Network_dataset_2.csv → 46 columns
✅ Network_dataset_20.csv → 46 columns
✅ Network_dataset_21.csv → 46 columns
✅ Network_dataset_22.csv → 46 columns
✅ Network_dataset_23.csv → 46 columns
✅ Network_dataset_3.csv → 46 columns
✅ Network_dataset_4.csv → 46 columns
✅ Network_dataset_5.csv → 46 columns
✅ Network_dataset_6.cs

In [22]:
import pandas as pd
import numpy as np
import joblib
import time
from sklearn.exceptions import NotFittedError

# ============================================================
# 🔧 CONFIGURATION
# ============================================================
MODEL_PATH = r"D:\RealTime_Alert_Analysis\report\training_report\universal_trained_model_fixed_v7.joblib"
SCALER_PATH = r"D:\RealTime_Alert_Analysis\report\training_report\universal_standard_scaler_fixed_v7.joblib"
CSV_PATH = r"D:\RealTime_Alert_Analysis\dataset\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"   # 👈 Change this to your test CSV

# ============================================================
# 🚀 LOAD MODEL + SCALER
# ============================================================
print(f"✅ Loading model: {MODEL_PATH}")
bundle = joblib.load(MODEL_PATH)

print(f"✅ Loading scaler: {SCALER_PATH}")
scaler = joblib.load(SCALER_PATH)

clf = bundle["classifier"]
iso = bundle["isolation_forest"]
le = bundle["label_encoder"]
model_features = bundle["feature_names"]

print(f"🧩 Model features: {len(model_features)}")

# ============================================================
# 📂 LOAD DATA
# ============================================================
print(f"📦 Loading dataset: {CSV_PATH}")
df = pd.read_csv(CSV_PATH, low_memory=False)
print(f"✅ CSV loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# ============================================================
# 🧹 CLEAN & ALIGN
# ============================================================
print("\n🧹 Cleaning dataset before prediction...")

# Keep only numeric columns
df = df.select_dtypes(include=[np.number])

# Align dataset columns to model’s feature order
df = df.reindex(columns=model_features, fill_value=np.nan)
missing_cols = set(model_features) - set(df.columns)
extra_cols = set(df.columns) - set(model_features)

print(f"🔍 Missing columns filled: {len(missing_cols)}")
print(f"🔍 Extra columns ignored: {len(extra_cols)}")

# Replace invalid numbers
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.astype(np.float32)
df.fillna(df.mean(numeric_only=True), inplace=True)

print(f"📊 Dataset ready: {df.shape}")

# ============================================================
# ⚙️ SCALING
# ============================================================
try:
    X_scaled = scaler.transform(df)
except NotFittedError:
    print("❌ Scaler not fitted — fitting on current dataset.")
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df)
    joblib.dump(scaler, SCALER_PATH)
    print(f"💾 New scaler saved at {SCALER_PATH}")

print(f"📏 Scaling complete — shape: {X_scaled.shape}")
print("   Scaled mean:", np.mean(X_scaled).round(5), 
      "std:", np.std(X_scaled).round(5))

# ============================================================
# 🧠 PREDICTION
# ============================================================
print("\n🤖 Running classification and anomaly detection...")

y_pred = clf.predict(X_scaled)
pred_labels = le.inverse_transform(y_pred)

# Optional anomaly detection (IsolationForest)
try:
    anomaly_flags = iso.predict(X_scaled)
    anomaly_rate = (anomaly_flags == -1).mean() * 100
except Exception as e:
    anomaly_rate = None
    print("⚠️ IsolationForest issue:", e)

# ============================================================
# 📊 RESULTS
# ============================================================
pred_series = pd.Series(pred_labels)
label_counts = pred_series.value_counts()

print("\n📈 Prediction Summary:")
print(label_counts)

if len(label_counts) == 1:
    print("\n⚠️ All records predicted as a single label — likely feature misalignment or zero variance after scaling.")
    print("   → Check dataset distribution and missing columns above.")
else:
    print("\n✅ Multiple labels detected — predictions seem valid.")

if anomaly_rate is not None:
    print(f"\n🧩 Anomaly Rate (IsolationForest): {anomaly_rate:.2f}%")

# ============================================================
# 💾 EXPORT RESULTS
# ============================================================
output_path = r"D:\RealTime_Alert_Analysis\report\predicted_output_v7.csv"
out_df = df.copy()
out_df["Predicted_Label"] = pred_series
if anomaly_rate is not None:
    out_df["Anomaly_Flag"] = anomaly_flags

out_df.to_csv(output_path, index=False)
print(f"\n💾 Results exported: {output_path}")

print("\n🎉 Prediction completed successfully!")


✅ Loading model: D:\RealTime_Alert_Analysis\report\training_report\universal_trained_model_fixed_v7.joblib
✅ Loading scaler: D:\RealTime_Alert_Analysis\report\training_report\universal_standard_scaler_fixed_v7.joblib
🧩 Model features: 197
📦 Loading dataset: D:\RealTime_Alert_Analysis\dataset\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
✅ CSV loaded: 225745 rows, 79 columns

🧹 Cleaning dataset before prediction...
🔍 Missing columns filled: 0
🔍 Extra columns ignored: 0
📊 Dataset ready: (225745, 197)
📏 Scaling complete — shape: (225745, 197)
   Scaled mean: nan std: nan

🤖 Running classification and anomaly detection...
⚠️ IsolationForest issue: Input X contains NaN.
IsolationForest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an impu

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhoogla/unswnb15")

print("Path to dataset files:", path)

d:\RealTime_Alert_Analysis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 11.7M/11.7M [00:02<00:00, 5.05MB/s]

Extracting files...


Path to dataset files: C:\Users\gange\.cache\kagglehub\datasets\dhoogla\unswnb15\versions\5


In [5]:
import pandas as pd

train = pd.read_parquet(r"D:\RealTime_Alert_Analysis\report\training_report\UNSW_NB15_training-set.parquet")
test = pd.read_parquet(r"D:\RealTime_Alert_Analysis\report\training_report\UNSW_NB15_testing-set.parquet")

print(train.shape)
print(train.columns)


(175341, 36)
Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt',
       'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
       'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
       'attack_cat', 'label'],
      dtype='object')


In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mrwellsdavid/unsw-nb15")

print("Path to dataset files:", path)

Resuming download from 60817408 bytes (95440229 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/mrwellsdavid/unsw-nb15?dataset_version_number=1 (60817408/156257637) bytes left.


100%|██████████| 149M/149M [00:08<00:00, 11.1MB/s] 

Extracting files...


Path to dataset files: C:\Users\gange\.cache\kagglehub\datasets\mrwellsdavid\unsw-nb15\versions\1


In [10]:
import os
import zipfile
import requests
import pandas as pd
from tqdm import tqdm
import subprocess

# ======================================================
# CONFIG
# ======================================================
DATA_DIR = r"C:\Users\gange\datasets"
CICIDS2017_URL = "https://www.kaggle.com/api/v1/datasets/download/cicdataset/cicids2017"
CICIDS2018_URL = "https://www.kaggle.com/api/v1/datasets/download/dhoogla/nfcsecicids2018v2"

OUT_2017 = os.path.join(DATA_DIR, "CICIDS2017.zip")
OUT_2018 = os.path.join(DATA_DIR, "CICIDS2018.zip")

EXTRACT_2017 = os.path.join(DATA_DIR, "CICIDS2017")
EXTRACT_2018 = os.path.join(DATA_DIR, "CICIDS2018")

OUTPUT_PARQUET = os.path.join(DATA_DIR, "cicids_combined.parquet")

# ======================================================
# Download with progress bar
# ======================================================
def download_file(url, output_path):
    print(f"📥 Downloading: {url}")

    headers = {}
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))

        with open(output_path, "wb") as f, tqdm(
            desc="Downloading",
            total=total,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in r.iter_content(chunk_size=8192):
                size = f.write(chunk)
                bar.update(size)

    print(f"✅ Saved: {output_path}")


# ======================================================
# Extract ZIP
# ======================================================
def extract_zip(src, dest):
    print(f"📦 Extracting {src} ...")
    with zipfile.ZipFile(src, 'r') as zip_ref:
        zip_ref.extractall(dest)
    print(f"✅ Extracted to: {dest}")


# ======================================================
# Clean and unify columns
# ======================================================
def normalize_cols(df):
    df.columns = (
        df.columns.astype(str)
        .str.lower()
        .str.strip()
        .str.replace('[^a-z0-9_]+', '_', regex=True)
        .str.replace('__+', '_', regex=True)
        .str.strip('_')
    )
    return df


# ======================================================
# Load CSVs inside folder recursively
# ======================================================
def load_all_csvs(folder):
    dfs = []
    for root, _, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(".csv"):
                fp = os.path.join(root, f)
                print(f"📄 Loading CSV: {fp}")
                try:
                    df = pd.read_csv(fp)
                    df = normalize_cols(df)
                    dfs.append(df)
                except Exception as e:
                    print("❌ Error:", e)
    return dfs


# ======================================================
# MAIN SCRIPT
# ======================================================
if __name__ == "__main__":
    print("🚀 CICIDS Downloader + Cleaner")

    # Create folder
    os.makedirs(DATA_DIR, exist_ok=True)

    # -----------------------------
    # DOWNLOAD DATASETS
    # -----------------------------
    print("\n==== CICIDS 2017 ====")
    download_file(CICIDS2017_URL, OUT_2017)
    extract_zip(OUT_2017, EXTRACT_2017)

    print("\n==== CICIDS 2018 ====")
    download_file(CICIDS2018_URL, OUT_2018)
    extract_zip(OUT_2018, EXTRACT_2018)

    # -----------------------------
    # LOAD DATA
    # -----------------------------
    print("\n📥 Loading all CICIDS 2017 datasets...")
    dfs_2017 = load_all_csvs(EXTRACT_2017)

    print("\n📥 Loading all CICIDS 2018 datasets...")
    dfs_2018 = load_all_csvs(EXTRACT_2018)

    all_dfs = dfs_2017 + dfs_2018
    print(f"📊 Total datasets loaded: {len(all_dfs)}")

    # -----------------------------
    # MERGE
    # -----------------------------
    print("🔄 Merging all datasets...")
    mega_df = pd.concat(all_dfs, ignore_index=True)

    print("🧹 Cleaning NaN...")
    mega_df.fillna(0, inplace=True)

    # -----------------------------
    # SAVE PARQUET
    # -----------------------------
    mega_df.to_parquet(OUTPUT_PARQUET)

    print("\n🎉 DONE!")
    print(f"💾 Combined dataset saved to:\n{OUTPUT_PARQUET}")


🚀 CICIDS Downloader + Cleaner

==== CICIDS 2017 ====
📥 Downloading: https://www.kaggle.com/api/v1/datasets/download/cicdataset/cicids2017


ConnectTimeout: HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/datasets/download/cicdataset/cicids2017 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028A052AC5E0>, 'Connection to www.kaggle.com timed out. (connect timeout=None)'))

In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ernie55ernie/improved-cicids2017-and-csecicids2018")

print("Path to dataset files:", path)

100%|██████████| 10.2G/10.2G [13:43<00:00, 13.3MB/s]  

Extracting files...


Path to dataset files: C:\Users\gange\.cache\kagglehub\datasets\ernie55ernie\improved-cicids2017-and-csecicids2018\versions\1


In [12]:
import pandas as pd

file_path = r"D:\RealTime_Alert_Analysis\dataset\CICIDS2017_improved\tuesday.csv"  # <- change this

df = pd.read_csv(file_path, nrows=5)  # read only first 5 rows (fast)

print("\n📌 Total columns:", len(df.columns))
print("\n📌 Column names:\n")
for col in df.columns:
    print(col)



📌 Total columns: 91

📌 Column names:

id
Flow ID
Src IP
Src Port
Dst IP
Dst Port
Protocol
Timestamp
Flow Duration
Total Fwd Packet
Total Bwd packets
Total Length of Fwd Packet
Total Length of Bwd Packet
Fwd Packet Length Max
Fwd Packet Length Min
Fwd Packet Length Mean
Fwd Packet Length Std
Bwd Packet Length Max
Bwd Packet Length Min
Bwd Packet Length Mean
Bwd Packet Length Std
Flow Bytes/s
Flow Packets/s
Flow IAT Mean
Flow IAT Std
Flow IAT Max
Flow IAT Min
Fwd IAT Total
Fwd IAT Mean
Fwd IAT Std
Fwd IAT Max
Fwd IAT Min
Bwd IAT Total
Bwd IAT Mean
Bwd IAT Std
Bwd IAT Max
Bwd IAT Min
Fwd PSH Flags
Bwd PSH Flags
Fwd URG Flags
Bwd URG Flags
Fwd RST Flags
Bwd RST Flags
Fwd Header Length
Bwd Header Length
Fwd Packets/s
Bwd Packets/s
Packet Length Min
Packet Length Max
Packet Length Mean
Packet Length Std
Packet Length Variance
FIN Flag Count
SYN Flag Count
RST Flag Count
PSH Flag Count
ACK Flag Count
URG Flag Count
CWR Flag Count
ECE Flag Count
Down/Up Ratio
Average Packet Size
Fwd Segment S

In [13]:
import os
import pandas as pd

# =============================
# CONFIG — SET YOUR PATHS HERE
# =============================
DATASETS = [
    r"D:\RealTime_Alert_Analysis\dataset\CICIDS2017_improved",
    r"D:\RealTime_Alert_Analysis\dataset\CSECICIDS2018_improved",
    r"D:\RealTime_Alert_Analysis\dataset\UNSW_NB15_training-set.csv",
    r"D:\RealTime_Alert_Analysis\dataset\UNSW_NB15_testing-set.csv",
]

OUTPUT_FILE = r"D:\RealTime_Alert_Analysis\dataset\column_report.txt"


def get_columns_from_csv(file_path):
    """Read only header of CSV to extract column names safely."""
    try:
        df = pd.read_csv(file_path, nrows=1)  # read only 1 row
        return list(df.columns)
    except Exception as e:
        return [f"ERROR reading file: {e}"]


def scan_datasets():
    report_lines = []
    report_lines.append("===== COLUMN REPORT =====\n\n")

    for path in DATASETS:

        if os.path.isdir(path):
            report_lines.append(f"\n📁 Folder: {path}\n")

            for fname in os.listdir(path):
                if fname.lower().endswith(".csv"):
                    full_path = os.path.join(path, fname)
                    report_lines.append(f"\n--- File: {fname} ---")
                    cols = get_columns_from_csv(full_path)
                    for c in cols:
                        report_lines.append(f"{c}")
        else:
            # Single CSV file
            report_lines.append(f"\n📄 File: {path}\n")
            cols = get_columns_from_csv(path)
            for c in cols:
                report_lines.append(f"{c}")

    # Save output
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        f.write("\n".join(report_lines))

    print(f"\n✅ Column report saved to: {OUTPUT_FILE}")


if __name__ == "__main__":
    scan_datasets()



✅ Column report saved to: D:\RealTime_Alert_Analysis\dataset\column_report.txt
